In [4]:
# 处理ml-1m
import json 
import pandas as pd
import os

data_dir = 'datasets/ml-1m/benchmark_proc_data/data/'

valid_df = pd.read_parquet(os.path.join('datasets/ml-1m/benchmark_proc_data/valid.parquet.gz'))
test_df = pd.read_parquet(os.path.join('datasets/ml-1m/benchmark_proc_data/test.parquet.gz'))
df = pd.concat([valid_df,test_df])
df = df.iloc[-100000:]
df = df.reset_index(drop=True)

import numpy as np
np.random.seed(42)
df['shared'] = np.random.choice([0,1,2,3],size=len(df))
train_df = df.iloc[:int(0.6*df.shape[0])]
print(train_df.shape[0])
train_df

60000


,timestamp,User ID,Gender,Age,Job,Zipcode,Movie ID,Movie title,Movie genre,rating,labels,user_hist,hist_rating,shared
0,978131240,25,male,18-24,college/grad student,01609,2700,"South Park: Bigger, Longer and Uncut (1999)",Animation,2,0,"[2253, 1198, 2628, 2028, 2683, 2355, 223, 2712...","[4, 5, 5, 5, 3, 3, 5, 4, 5, 4, 4, 5, 5, 4, 4, ...",2
1,978131266,25,male,18-24,college/grad student,01609,2693,Trekkies (1997),Documentary,3,0,"[2253, 1198, 2628, 2028, 2683, 2355, 223, 2712...","[4, 5, 5, 5, 3, 3, 5, 4, 5, 4, 4, 5, 5, 4, 4, ...",3
2,978131266,25,male,18-24,college/grad student,01609,3114,Toy Story 2 (1999),Animation,4,1,"[2253, 1198, 2628, 2028, 2683, 2355, 223, 2712...","[4, 5, 5, 5, 3, 3, 5, 4, 5, 4, 4, 5, 5, 4, 4, ...",0
3,978131277,48,male,25-34,college/grad student,92107,2288,"Thing, The (1982)",Action,3,0,"[1197, 569, 66, 1193, 2054, 2152, 552, 3897, 3...","[3, 2, 1, 4, 2, 1, 3, 4, 4, 4, 3, 3, 3, 3, 5, ...",2
4,978131283,25,male,18-24,college/grad student,01609,2987,Who Framed Roger Rabbit? (1988),Adventure,4,1,"[2253, 1198, 2628, 2028, 2683, 2355, 223, 2712...","[4, 5, 5, 5, 3, 3, 5, 4, 5, 4, 4, 5, 5, 4, 4, ...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,998322129,3085,male,25-34,college/grad student,78703,593,"Silence of the Lambs, The (1991)",Drama,5,1,"[1210, 1704, 3462, 509, 2028, 3148, 3354, 3408...","[5, 4, 3, 3, 5, 5, 4, 4, 3, 4, 2, 3, 4]",0
59996,998322158,3085,male,25-34,college/grad student,78703,1036,Die Hard (1988),Action,5,1,"[1210, 1704, 3462, 509, 2028, 3148, 3354, 3408...","[5, 4, 3, 3, 5, 5, 4, 4, 3, 4, 2, 3, 4, 5]",0
59997,998322165,1597,male,25-34,programmer,10002,1449,Waiting for Guffman (1996),Comedy,4,1,"[1210, 1834, 2348, 6, 1230, 3176, 3893, 3949, ...","[4, 3, 4, 2, 4, 5, 5, 4, 1, 5, 4, 4, 5, 4, 4, ...",3
59998,998322165,1597,male,25-34,programmer,10002,593,"Silence of the Lambs, The (1991)",Drama,4,1,"[1210, 1834, 2348, 6, 1230, 3176, 3893, 3949, ...","[4, 3, 4, 2, 4, 5, 5, 4, 1, 5, 4, 4, 5, 4, 4, ...",2


In [ ]:
import json 
import pandas as pd
import os

data_dir = 'datasets/ml-1m/benchmark_proc_data/data/'

with open(data_dir+'rella_valid/valid_10_simple.json', 'r') as f:
    rella_train_data = json.load(f)
with open(data_dir+'rella_test/test_10_simple.json', 'r') as f:
    rella_test_data = json.load(f)
rella_data = rella_train_data + rella_test_data
length = len(rella_data)
# print("rella data length: ", length)
rella_data = rella_data[-100000:]
length = len(rella_data)
# print("rella data length: ", length)

train_data, valid_data, test_data = rella_data[:int(0.6*length)], rella_data[int(0.6*length):int(0.8*length)], rella_data[int(0.8*length):]
# print(len(train_data), len(valid_data), len(test_data))
with open(data_dir+f'train/train_10_simple.json', 'w') as f:
    json.dump(train_data, f)
with open(data_dir+f'valid/valid_10_simple.json', 'w') as f:
    json.dump(valid_data, f)
with open(data_dir+f'test/test_10_simple.json', 'w') as f:
    json.dump(test_data, f)

In [3]:
with open(data_dir+'train/train_10_simple.json', 'r') as f:
    train_data = json.load(f)
length = len(train_data)
print(length)

unique_user_list = train_df['User ID'].unique()
user_num = len(unique_user_list)
print(user_num)

import random
random.seed(42)
random.shuffle(unique_user_list)

# 将train进一步划分成 retain data 和 forget data
for p in [0.2]:
    forget_user = unique_user_list[:int(p*user_num)]
    # print(forget_user)
    # print(len(forget_user))
    selected_index = train_df[train_df['User ID'].isin(forget_user)].index
    selected_index = selected_index.tolist()
    # print(len(selected_index))
    forget_data = [record for idx, record in enumerate(train_data) if idx in selected_index ]
    retain_data = [record for idx, record in enumerate(train_data) if idx not in selected_index  ]
    print(len(forget_data), len(retain_data))
    with open(data_dir+f'train/retain_{p}_user_10_simple.json', 'w') as f:
        json.dump(retain_data, f)
    with open(data_dir+f'train/forget_{p}_user_10_simple.json', 'w') as f:
        json.dump(forget_data, f)


60000
1025
['3196' '987' '1019' '5424' '1202' '1501' '13' '1447' '2683' '2814' '934'
 '1557' '2749' '3916' '910' '231' '5281' '1930' '2272' '116' '3441' '3315'
 '5949' '5493' '2991' '1699' '3313' '33' '5520' '1389' '4361' '1794'
 '1088' '716' '962' '2127' '1836' '795' '5891' '5972' '1142' '4070' '2454'
 '3685' '774' '2757' '1912' '150' '3503' '6040' '2791']
51
4000
4000 56000
['3196' '987' '1019' '5424' '1202' '1501' '13' '1447' '2683' '2814' '934'
 '1557' '2749' '3916' '910' '231' '5281' '1930' '2272' '116' '3441' '3315'
 '5949' '5493' '2991' '1699' '3313' '33' '5520' '1389' '4361' '1794'
 '1088' '716' '962' '2127' '1836' '795' '5891' '5972' '1142' '4070' '2454'
 '3685' '774' '2757' '1912' '150' '3503' '6040' '2791' '3391' '3971' '78'
 '65' '3717' '583' '3125' '770' '1058' '5329' '124' '355' '609' '3150'
 '2390' '382' '4940' '1519' '224' '1324' '2409' '4738' '1974' '3' '5918'
 '462' '3519' '4674' '3411' '5548' '2944' '1252' '707' '5557' '2593'
 '5605' '299' '899' '22' '1029' '2478' '1

In [5]:
import json
data_dir = 'datasets/ml-1m/benchmark_proc_data/data/'
with open(data_dir+'train/train_10_simple.json', 'r') as f:
    train_data = json.load(f)
length = len(train_data)
print(length)

unique_user_list = train_df['User ID'].unique()
user_num = len(unique_user_list)
print(user_num)

import random
random.seed(42)
random.shuffle(unique_user_list)

# sisa方法：每个shard去掉forget data
for shared in [0,1,2,3]:
    for p in [0.2]:
        forget_user = unique_user_list[:int(p*user_num)]
        print(len(forget_user))
        shared_index = train_df[train_df['shared'] == shared].index
        shared_index = shared_index.tolist()
        
        forget_index = train_df[train_df['User ID'].isin(forget_user)].index
        forget_index = forget_index.tolist()

        retain_data = [record for idx, record in enumerate(train_data) if idx not in forget_index and idx in shared_index]
        print(len(retain_data))
        # 保存每个shared中去掉forget data之后的数据
        with open(data_dir+f'sisa/shared_{shared}_{p}_user_10_simple.json', 'w') as f:
            json.dump(retain_data, f)


60000
1025
205
11770
205
11738
205
11761
205
11749


In [ ]:
# 划分eraser
import json
data_dir = '/data/hangyuwang/unlearning/datasets/ml-1m/benchmark_proc_data/data/'
with open(data_dir+'train/train_10_simple.json', 'r') as f:
    train_data = json.load(f)
length = len(train_data)
print('train data length:', length)

unique_user_list = train_df['User ID'].unique()
user_num = len(unique_user_list)
print('train data user num:', user_num)

import random
random.seed(42)
random.shuffle(unique_user_list)

for p in [0.2]:
    for shared in [0,1,2,3]:
        forget_user = unique_user_list[:int(p*user_num)]
        # print(len(forget_user))
        retain_user = unique_user_list[int(p*user_num):]
        # print(len(retain_user))
        shared_user = retain_user[int(len(retain_user)*shared/4):int(len(retain_user)*(shared+1)/4)]
        print(len(shared_user))
        shared_index = train_df[train_df['User ID'].isin(shared_user)].index
        shared_index = shared_index.tolist()
        
        forget_index = train_df[train_df['User ID'].isin(forget_user)].index
        forget_index = forget_index.tolist()

        retain_data = [record for idx, record in enumerate(train_data) if idx not in forget_index and idx in shared_index]
        print(len(retain_data))
        # 保存每个shared中去掉forget data之后的数据
        with open(data_dir+f'sisa/eraser_{shared}_{p}_user_10_simple.json', 'w') as f:
            json.dump(retain_data, f)
